In [1]:
import os
import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
from torchvision.models import vgg16, VGG16_Weights
import torchvision.transforms.v2 as T

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class labels
DATA_LABELS = [
    "freshapples", "freshbanana", "freshoranges",
    "rottenapples", "rottenbanana", "rottenoranges"
]
N_CLASSES = len(DATA_LABELS)
BATCH_SIZE = 32


In [3]:
# Transforms for training and validation
train_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(20),
    T.ColorJitter(0.3, 0.3, 0.3),
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True),
    T.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225))
])

val_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True),
    T.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225))
])

In [4]:
# Custom dataset class
class FruitDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.imgs = []
        self.labels = []
        self.transform = transform

        for idx, label in enumerate(DATA_LABELS):
            img_paths = glob.glob(os.path.join(root_dir, label, "*.png"))
            for path in img_paths:
                self.imgs.append(path)
                self.labels.append(idx)

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        image = read_image(self.imgs[idx], ImageReadMode.RGB)
        image = self.transform(image)
        label = self.labels[idx]
        return image, torch.tensor(label)


In [5]:
# Data loading paths
train_path = "C:/Users/okeiy/Downloads/Nvdia Learning/dataset/train"
val_path = "C:/Users/okeiy/Downloads/Nvdia Learning/dataset/test"

train_dataset = FruitDataset(train_path, train_transforms)
val_dataset = FruitDataset(val_path, val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [6]:
# Build model using VGG16 as base
vgg_model = vgg16(weights=VGG16_Weights.DEFAULT)
vgg_model.requires_grad_(True)

model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Dropout(0.3),
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(500, N_CLASSES)
).to(device)


In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    print(f"Train - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [8]:
# Validation loop
def validate(model, loader, loss_fn, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)

            total_loss += loss.item() * images.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    print(f"Validation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [9]:
# Run training
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train(model, train_loader, criterion, optimizer, device)
    validate(model, val_loader, criterion, device)


Epoch 1/10
Train - Loss: 0.1778, Accuracy: 94.01%
Validation - Loss: 0.0565, Accuracy: 98.41%

Epoch 2/10
Train - Loss: 0.0645, Accuracy: 97.99%
Validation - Loss: 0.0193, Accuracy: 99.48%

Epoch 3/10
Train - Loss: 0.0310, Accuracy: 99.16%
Validation - Loss: 0.0618, Accuracy: 98.41%

Epoch 4/10
Train - Loss: 0.0495, Accuracy: 98.65%
Validation - Loss: 0.1756, Accuracy: 96.26%

Epoch 5/10
Train - Loss: 0.0330, Accuracy: 99.15%
Validation - Loss: 0.0027, Accuracy: 99.93%

Epoch 6/10
Train - Loss: 0.0353, Accuracy: 99.09%
Validation - Loss: 0.0672, Accuracy: 98.37%

Epoch 7/10
Train - Loss: 0.0237, Accuracy: 99.39%
Validation - Loss: 0.0026, Accuracy: 99.96%

Epoch 8/10
Train - Loss: 0.0462, Accuracy: 98.87%
Validation - Loss: 0.0290, Accuracy: 99.33%

Epoch 9/10
Train - Loss: 0.0330, Accuracy: 99.18%
Validation - Loss: 0.0257, Accuracy: 99.11%

Epoch 10/10
Train - Loss: 0.0147, Accuracy: 99.60%
Validation - Loss: 0.0111, Accuracy: 99.67%


In [10]:
# Save model
torch.save(model.state_dict(), "fruit_model_1.pth")